In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generate queries with neurocard

See neurocard.ipynb in https://github.com/Erostrate9/neurocard

In [ ]:
!cp /content/drive/MyDrive/job-light-content-2500.csv /content/job-light-content-2500.csv

- Notice: As mentioned in Fauce paper, JOB-base contains the queries in JOB-base generated based on numeric columns in JOB-light.
- So instead of any String column, there're only int and float columns in the training data.
- There're only 6 tables in JOB-base:
  1. title
  2. cast_info
  3. movie_info
  4. movie_company
  5. movie_keyword
  6. movie_info_idx


NeuroCard is highly time-consuming. We don't use its generated queries for training Fauce. We use JobLight-train in https://github.com/andreaskipf/learnedcardinalities/blob/master/data/train.csv

# Dataset overview

In [ ]:
df = pd.read_csv("/content/vldb2021_fauce/training_data/raw_data/JobLight-train.csv", sep='#', names=['Tables','Joins','Filters','Cardinality'])
df

,Tables,Joins,Filters,Cardinality
0,"title,movie_info_idx",title.id=movie_info_idx.movie_id,"title.kind_id,=,7,movie_info_idx.info_type_id,...",283812
1,title,NaN,"title.production_year,>,2004",1107925
2,movie_info,NaN,"movie_info.info_type_id,<,4",3624977
3,"title,movie_companies",title.id=movie_companies.movie_id,"movie_companies.company_id,<,27",134807
4,movie_keyword,NaN,"movie_keyword.keyword_id,<,55",54826
...,...,...,...,...
99995,"title,movie_keyword",title.id=movie_keyword.movie_id,"movie_keyword.keyword_id,>,1951",3102738
99996,"title,movie_companies,movie_keyword","title.id=movie_companies.movie_id,title.id=mov...","title.kind_id,<,6,movie_companies.company_type...",738
99997,"title,movie_info,movie_info_idx","title.id=movie_info.movie_id,title.id=movie_in...","title.kind_id,=,7,title.production_year,=,2006...",103764
99998,"title,movie_info_idx,movie_keyword","title.id=movie_info_idx.movie_id,title.id=movi...","title.kind_id,=,7,title.production_year,<,1990...",9


In [ ]:
# how many tables each queries contains
df['Table_Count'] = df['Tables'].str.split(',').apply(len)
stats = df['Table_Count'].agg(['min', 'max'])
print(stats)
all_tables = df['Tables'].str.split(',').explode()
print(all_tables.unique())

min    1
max    3
Name: Table_Count, dtype: int64
['title' 'movie_info_idx' 'movie_info' 'movie_companies' 'movie_keyword'
 'cast_info']


In [ ]:
def count_columns
# how many columns involved in joins
df['Joins'] = df['Joins'].fillna('')
df['Join_Count'] = df['Joins'].str.split(',').apply(len)
stats = df['Join_Count'].agg(['min', 'max'])
print(stats)
all_columns = df['Joins'].str.split(',').explode()
print(all_columns.unique())
# join relationship: title.id and movie_id (PK and FK)

min    1
max    2
Name: Join_Count, dtype: int64
['title.id=movie_info_idx.movie_id' '' 'title.id=movie_companies.movie_id'
 'title.id=movie_info.movie_id' 'title.id=cast_info.movie_id'
 'title.id=movie_keyword.movie_id']


In [ ]:
# how many predicates involved in Filters
df['Filters'] = df['Filters'].fillna('')
df['Filter_Count'] = df['Filters'].apply(lambda x: len(x.split(',')) // 3 if x else 0)
stats=df['Filter_Count'].agg(['min', 'max'])
print(stats)

min    0
max    6
Name: Filter_Count, dtype: int64


In [ ]:
df['Filters'] = df['Filters'].fillna('')
columns = df['Filters'].apply(lambda x: [x.split(',')[i] for i in range(0, len(x.split(',')), 3)])
unique_columns = set(columns.explode())
print(unique_columns)

{'', 'movie_companies.company_id', 'cast_info.person_id', 'title.kind_id', 'cast_info.role_id', 'movie_keyword.keyword_id', 'movie_companies.company_type_id', 'movie_info.info_type_id', 'movie_info_idx.info_type_id', 'title.production_year'}


In [ ]:
stats = df['Cardinality'].agg(['min', 'max'])
print(stats)

min            1
max    460456073
Name: Cardinality, dtype: int64


# Queries Featurization

## Clone GitHithub repository

In [2]:
!git config --global user.email ericsun42@outlook.com
!git config --global user.name Erostrate9
!mkdir -p /root/.ssh && cp -r "/content/drive/My Drive/ssh/." /root/.ssh/
!ssh -T git@github.com


Hi Erostrate9! You've successfully authenticated, but GitHub does not provide shell access.


In [3]:
!git clone -b eric git@github.com:Erostrate9/vldb2021_fauce.git

Cloning into 'vldb2021_fauce'...
remote: Enumerating objects: 1181, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 1181 (delta 145), reused 286 (delta 136), pack-reused 884 (from 1)
Receiving objects: 100% (1181/1181), 287.05 MiB | 25.86 MiB/s, done.
Resolving deltas: 100% (561/561), done.
Updating files: 100% (136/136), done.


In [ ]:
%%bash
cp "/content/drive/MyDrive/Colab Notebooks/Fauce.ipynb" /content/vldb2021_fauce/colab_notebooks/Fauce.ipynb
cd /content/vldb2021_fauce
git add .
git commit -m "Update test results"
git push origin eric

[eric 522a9c6] Update test results
 9 files changed, 30188 insertions(+), 104491 deletions(-)
 rewrite colab_notebooks/Fauce.ipynb (65%)
 create mode 100644 models_training/datasets/job-light-content-2500.csv
 delete mode 100644 models_training/datasets/normalized_JobLight-train.csv
 delete mode 100644 models_training/datasets/normalized_job-light-content-2500.csv
 delete mode 100644 models_training/datasets/normalized_job-light.csv
 create mode 100644 models_training/get-pip.py
 create mode 100644 models_training/results/trained_on_JobLight-train_tested_on_job-light-content-2500.csv
 create mode 100644 training_data/featurization/job-light-content-2500.csv
 delete mode 100644 training_data/featurization/neurocard.csv


To github.com:Erostrate9/vldb2021_fauce.git
   26813b7..522a9c6  eric -> eric


In [ ]:
# Divyanshu's works (Optional)
!mkdir py3
!cd py3 && git clone -b dverma/update_fauce git@github.com:Erostrate9/vldb2021_fauce.git

Cloning into 'vldb2021_fauce'...
remote: Enumerating objects: 757, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 757 (delta 117), reused 269 (delta 84), pack-reused 442 (from 1)
Receiving objects: 100% (757/757), 4.75 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (337/337), done.


## Install environment

In [ ]:
!apt install build-essential
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install --channel defaults conda --yes
!conda update --channel defaults --all --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2024-08-27 05:14:41--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148981743 (142M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 142.08M   249MB/s    in 0.6s    

2024-08-27 05:14:42 (249 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [148981743/148981743]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
install

In [ ]:
%%bash
conda env create -f /content/vldb2021_fauce/queries_featurization/environment.yml
curl https://bootstrap.pypa.io/pip/2.7/get-pip.py -o get-pip.py
source activate qf
python get-pip.py --force-reinstall
pip install networkx==1.11
pip install numpy==1.11.2
pip install gensim==0.12.1
pip install tensorflow==0.12.1
pip install joblib==0.11
pip install scikit-learn
pip install singledispatch
pip install pandas

Channels:
 - conda-forge
 - defaults
 - prometeia
 - legel
 - free
Platform: linux-64
Solving environment: ...working... done

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate qf
#
# To deactivate an active environment, use
#
#     $ conda deactivate

  Attempting uninstall: networkx
    Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2
  Attempting uninstall: joblib
    Found existing installation: joblib 0.14.1
    Uninstalling joblib-0.14.1:
      Successfully uninstalled joblib-0.14.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.11.2
    Uninstalling numpy-1.11.2:
      Successfully uninstalled numpy-1.11.2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1863k  100 1863k    0     0  6667k      0 --:--:-- --:--:-- --:--:-- 6655k


In [ ]:
# in colab
!pip install scikit-learn
!pip install pandas
!pip install tensorflow

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 33.3 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 62.7 MB/s eta 0:00:00
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 87.2 MB/s eta 0:00:00
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)


## Table Encoding using a graph embedding method

- The whole schema of IMDB database is shown in the following figure.
However, there're only 6 tables in Job-light:
  1. title
  2. cast_info
  3. movie_info
  4. movie_company
  5. movie_keyword
  6. movie_info_idx
![IMDB schema](https://www.researchgate.net/profile/Peter-Boncz/publication/319893076/figure/fig2/AS:631637725438007@1527605577677/MDB-schema-with-key-foreign-key-relationships-Underlined-attributes-are-primary-keys.png)

- Fauce paper didn't record full details of training data. They didn't mention which 15 tables they used, and the edgelist they offered doesn't correspond to the IMDB schema.
- So I use the Job-light (JOB-base) version with 6 tables.
```csv
title 1
cast_info 2
movie_info 3
movie_companies 4
movie_keyword 5
movie_info_idx 6
```



In [ ]:
# m =6, dimensions for each table = [log(m+1)] = 3
%%bash
cd /content/vldb2021_fauce/queries_featurization/graph_embedding
source activate qf
python main.py --input graph/graph.edgelist --output emb/graph_node.emd --dimensions 3

Walk iteration:
1 / 3
2 / 3
3 / 3


/usr/local/envs/qf/lib/python2.7/site-packages/gensim/models/word2vec.py:651: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


The first line has the following format:

`num_of_nodes dim_of_representation`

The next lines are as follows:

`node_id dim1 dim2 ... dimd`
where dim1, ... , dimd is the d-dimensional representation learned by the graoh embedding method.

In [ ]:
# According to the PK and FK relationships, we build the edgelist
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/graph.edgelist

1 2
1 3
1 4
1 5
1 6

In [ ]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/emb/graph_node.emd

6 3
1 -0.180520 -0.171498 0.166586
6 0.049148 -0.038398 0.034771
3 -0.142707 0.037522 0.148151
2 0.134461 -0.017551 -0.034228
5 0.026742 -0.142876 -0.167332
4 -0.002204 0.161061 0.000822


## Join Encoding using Joins2Vec

In [ ]:
# !mkdir -p /content/vldb2021_fauce/queries_featurization/example_data/datasets/node_edges
# !touch /content/vldb2021_fauce/queries_featurization/example_data/datasets/node_edges/0.gexf
# !touch /content/vldb2021_fauce/queries_featurization/example_data/datasets/node.Labels

In [ ]:
!cat /content/vldb2021_fauce/queries_featurization/Joins2Vec/example_data/datasets/node_edges/0.gexf

<?xml version='1.0' encoding='utf-8'?>
<gexf version="1.1" xmlns="http://www.gexf.net/1.1draft" xmlns:viz="http://www.gexf.net/1.1draft/viz" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/XMLSchema-instance">
  <graph defaultedgetype="undirected" mode="static">
    <attributes class="node" mode="static">
      <attribute id="0" title="Label" type="string" />
    </attributes>
    <nodes>
      <node id="1" label="1">
        <attvalues>
          <attvalue for="0" value="1" />
        </attvalues>
      </node>
      <node id="2" label="2">
        <attvalues>
          <attvalue for="0" value="2" />
        </attvalues>
      </node>
      <node id="3" label="3">
        <attvalues>
          <attvalue for="0" value="3" />
        </attvalues>
      </node>
      <node id="4" label="4">
        <attvalues>
          <attvalue for="0" value="4" />
        </attvalues>
      </node>
      <node id="5" label="5">
        <attvalues>
     

Two-node graphs (d=1): 5 (AB, AC, AD, AE, AF)
Three-node graphs (d=2): 10 (ABC, ABD, ABE, ABF, ACD, ACE, ACF, ADE, ADF, AEF)
Four-node graphs (d=3): 10 (ABCD, ABCE, ABCF, ABDE, ABDF, ABEF, ACDE, ACDF, ACEF, ADEF)
Five-node graphs (d=4): 5 (ABCDE, ABCDF, ABCEF, ABDEF, ACDEF)
Five-node graphs (d=5): 1 (ABCDEF)

n = the number of possible join relationships among the database tables = 5

In [ ]:
%%bash
source activate qf
cd /content/vldb2021_fauce/queries_featurization/Joins2Vec
# mkdir ../embeddings
python main.py --corpus ./example_data/datasets/node_edges --class_labels_file_name ./example_data/datasets/node.Labels --output_dir ./embeddings -d 5 --wlk_h 2

Device mapping: no known devices.
gradients/nce_loss/embedding_lookup_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/embedding_lookup_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/embedding_lookup_1_grad/strided_slice: (StridedSlice): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_2_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_1_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/Slice_3_grad/pack: (Pack): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/sub_1_grad/Shape_1: (Const): /job:localhost/replica:0/task:0/cpu:0
gradients/nce_loss/truediv_grad/Shape_1: (Const): /job:localhost/replica:0/task:0/cpu:0
gradients/Mean_grad/Prod_1: (Prod): /job:localhost/replica:0/task:0/cpu:0
gradients/Mean_grad/Maximum: (Maximum): /job:localhost/replic

INFO:root:Loaded 1 graph file names form ./example_data/datasets/node_edges
INFO:root:Dumped subgraph2vec sentences for all 1 graphs in ./example_data/datasets/node_edges in 0.0 sec
INFO:root:Initializing SKIPGRAM...
INFO:root:vocabulary size: 19
INFO:root:number of documents: 1
INFO:root:number of words to be trained: 20
I tensorflow/core/common_runtime/direct_session.cc:255] Device mapping:

I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/nce_loss/embedding_lookup_grad/strided_slice: (StridedSlice)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/nce_loss/Slice_grad/pack: (Pack)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/embedding_lookup_grad/strided_slice: (StridedSlice)/job:localhost/replica:0/task:0/cpu:0
I tensorflow/core/common_runtime/simple_placer.cc:827] gradients/nce_loss/embedding_lookup_1_grad/strided_slice: (StridedSlice)/job:localhost/replica:0

CalledProcessError: Command 'b'source activate qf\ncd /content/vldb2021_fauce/queries_featurization/Joins2Vec\n# mkdir ../embeddings\npython main.py --corpus ./example_data/datasets/node_edges --class_labels_file_name ./example_data/datasets/node.Labels --output_dir ./embeddings -d 5 --wlk_h 2\n'' returned non-zero exit status 1.

maximum of allowed joins in a query = 5

Two-node graphs (d=1): 5 (AB, AC, AD, AE, AF)
- AB: `B#A`
- AC: `C#A`
- AD: `D#A`
- AE: `E#A`
- AF: `F#A`

Three-node graphs (d=2): 10 (ABC, ABD, ABE, ABF, ACD, ACE, ACF, ADE, ADF, AEF)

Four-node graphs (d=3): 10 (ABCD, ABCE, ABCF, ABDE, ABDF, ABEF, ACDE, ACDF, ACEF, ADEF)

Five-node graphs (d=4): 5 (ABCDE, ABCDF, ABCEF, ABDEF, ACDEF)

Five-node graphs (d=5): 6 (ABCDEF)

n = 5

## Columns Encoding

### Build local columns-dependency graphs
- We calculate the Randomized Dependence Coefficient (RDC) values for each pair
of columns in each table $T_i$ from the database $D$. If the RDC value
for two columns exceeds a threshold $𝜏$, then those two columns are
dependent with each other; otherwise, they are independent.
  - The RDC is a measure of nonlinear dependence between two variables.
- we set $𝜏$ as 0.4
- Once there exists a connection
(i.e., an edge) between two columns (i.e., vertices), the graph shows
those two columns are correlated.

Given the global columns-dependency graph, use the same graph embedding method for Table Encoding to transform "Columns" into vectors. Currently, Fauce only focuses on the columns with real-valued numbers.

To build the global columns-dependency graph, it includes 3 steps.
1. For each "Table" in the database, we calculate the Randomized Dependence Coefficient(RDC) values for each pair of "Columns" in each "Table" to build the local columns-dependency graph;
2. Connect the local columns-dependency graph based on the PK and FK relationships among "Tables" to build the global columns-dependency graph;
3. Given the global columns-dependency graph as the input, use the same graph embedding method (the method to represent "Tables" as vectors) to get the embeddings for the "Columns" in the database.

In [ ]:
!cd /content/vldb2021_fauce/queries_featurization/build_graphs && source activate qf && python setup.py install
!cd /content/vldb2021_fauce/queries_featurization/build_graphs && python setup.py install

In [ ]:
%%bash
git clone -b eric  https://github.com/Erostrate9/neurocard.git
chmod +x neurocard/neurocard/scripts/download_imdb.sh
cd neurocard
conda env create -f environment.yml
source activate neurocard
neurocard/scripts/download_imdb.sh

/content/neurocard/datasets/job /content/neurocard
aka_name.csv
aka_title.csv
cast_info.csv
char_name.csv
company_name.csv
company_type.csv
comp_cast_type.csv
complete_cast.csv
info_type.csv
keyword.csv
kind_type.csv
link_type.csv
movie_companies.csv
movie_info.csv
movie_info_idx.csv
movie_keyword.csv
movie_link.csv
name.csv
person_info.csv
role_type.csv
schematext.sql
title.csv
/content/neurocard


fatal: destination path 'neurocard' already exists and is not an empty directory.

CondaValueError: prefix already exists: /usr/local/envs/neurocard

+ mkdir -p datasets/job
+ pushd datasets/job
+ wget -c http://homepages.cwi.nl/~boncz/job/imdb.tgz
--2024-08-23 01:33:05--  http://homepages.cwi.nl/~boncz/job/imdb.tgz
Resolving homepages.cwi.nl (homepages.cwi.nl)... 192.16.191.44
Connecting to homepages.cwi.nl (homepages.cwi.nl)|192.16.191.44|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://homepages.cwi.nl/~boncz/job/imdb.tgz [following]
--2024-08-23 01:33:05--  https://homepages.cwi.nl/~boncz/job/imdb.tgz
Connecting to homepages.cwi.nl (homepages.cwi.nl)|192.16.191.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1263193115 (1.2G) [application/x-gzip]
Saving to: ‘imdb.tgz’

     0K .......... .......... .......... .......... ..........  0%  175K 1h57m
    50K .......... .......... .......... .......... ..........  0%  348

CalledProcessError: Command 'b'git clone -b eric  https://github.com/Erostrate9/neurocard.git\nchmod +x neurocard/neurocard/scripts/download_imdb.sh\ncd neurocard\nconda env create -f environment.yml\nsource activate neurocard\nneurocard/scripts/download_imdb.sh\n'' returned non-zero exit status 2.

In [ ]:
%%bash
source activate neurocard
cd neurocard
python neurocard/scripts/prepend_imdb_headers.py

I0823 02:29:45.595426 137148563261248 prepend_imdb_headers.py:76] Prepending header to datasets/job/aka_name.csv: id,person_id,name,imdb_index,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
I0823 02:29:46.361639 137148563261248 prepend_imdb_headers.py:76] Prepending header to datasets/job/aka_title.csv: id,movie_id,title,imdb_index,kind_id,production_year,phonetic_code,episode_of_id,season_nr,episode_nr,note,md5sum
I0823 02:29:46.801690 137148563261248 prepend_imdb_headers.py:76] Prepending header to datasets/job/cast_info.csv: id,person_id,movie_id,person_role_id,note,nr_order,role_id
I0823 02:30:04.570535 137148563261248 prepend_imdb_headers.py:76] Prepending header to datasets/job/char_name.csv: id,name,imdb_index,imdb_id,name_pcode_nf,surname_pcode,md5sum
I0823 02:30:06.948888 137148563261248 prepend_imdb_headers.py:76] Prepending header to datasets/job/company_name.csv: id,name,country_code,imdb_id,name_pcode_nf,name_pcode_sf,md5sum
I0823 02:30:07.134863 137148563261248 prepend_

In [ ]:
%%bash
mkdir /content/vldb2021_fauce/queries_featurization/build_graphs/dataset
cd /content/vldb2021_fauce/queries_featurization/build_graphs/dataset
cp /content/neurocard/datasets/job/title.csv title.csv
cp /content/neurocard/datasets/job/cast_info.csv cast_info.csv
cp /content/neurocard/datasets/job/movie_info.csv movie_info.csv
cp /content/neurocard/datasets/job/movie_companies.csv movie_companies.csv
cp /content/neurocard/datasets/job/movie_keyword.csv movie_keyword.csv
cp /content/neurocard/datasets/job/movie_info_idx.csv movie_info_idx.csv

In [ ]:
import pandas as pd
df_title = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv", escapechar='\\')
df_title = df_title[['id', 'kind_id', 'production_year']]
df_title = df_title.dropna()
df_title.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv", index=False)
df_title

<ipython-input-121-d2c38e78dd37>:2: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv", escapechar='\\')


,id,kind_id,production_year
0,80889,7,1980.0
1,5156,7,2010.0
2,197772,7,1962.0
3,111913,7,2012.0
5,40704,7,1971.0
...,...,...,...
2528295,2528270,1,1953.0
2528299,2528296,2,1985.0
2528305,2528298,3,2012.0
2528306,2528268,1,1942.0


In [ ]:
df_ci = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/cast_info.csv", escapechar='\\')
df_ci = df_ci[['person_id', 'movie_id', 'role_id']]
df_ci = df_ci.dropna()
df_ci.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/cast_info.csv", index=False)

In [ ]:
df_mi = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info.csv", escapechar='\\')
df_mi = df_mi[['movie_id', 'info_type_id']]
df_mi = df_mi.dropna()
df_mi.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info.csv", index=False)
df_mi

<ipython-input-123-91568cb19b2d>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mi = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info.csv", escapechar='\\')


,movie_id,info_type_id
0,921521,4
1,921522,4
2,921523,4
3,921524,4
4,921525,4
...,...,...
14835715,921516,4
14835716,921517,4
14835717,921518,4
14835718,921519,4


In [ ]:
df_mc = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_companies.csv", escapechar='\\')
df_mc = df_mc[['movie_id', 'company_id', 'company_type_id']]
df_mc = df_mc.dropna()
df_mc.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_companies.csv", index=False)
df_mc

,movie_id,company_id,company_type_id
0,2,1,1
1,2,1,1
2,11,2,1
3,44,3,1
4,50,4,1
...,...,...,...
2609124,2525599,28437,2
2609125,2525600,140876,2
2609126,2525601,234997,2
2609127,2525602,168049,2


In [ ]:
df_mk = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_keyword.csv", escapechar='\\')
df_mk = df_mk[['movie_id', 'keyword_id']]
df_mk = df_mk.dropna()
df_mk.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_keyword.csv", index=False)
df_mk

,movie_id,keyword_id
0,2,1
1,11,2
2,22,2
3,44,3
4,24,2
...,...,...
4523925,2525602,8621
4523926,2525602,5761
4523927,2525602,24715
4523928,2525602,8386


In [ ]:
df_mi_idx = pd.read_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info_idx.csv", escapechar='\\')
df_mi_idx = df_mi_idx[['movie_id','info_type_id']]
df_mi_idx = df_mi_idx.dropna()
df_mi_idx.to_csv("/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/movie_info_idx.csv", index=False)
df_mi_idx

,movie_id,info_type_id
0,2,99
1,2,100
2,2,101
3,11,99
4,11,100
...,...,...
1380030,2207311,113
1380031,1730276,113
1380032,2378686,113
1380033,2322570,113


In [ ]:
# !cp -r /content/vldb2021_fauce/queries_featurization/build_graphs/dataset /content/drive/MyDrive/datasets

In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install scipy
!pip install pandas

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 29.0 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [ ]:
!cp -r /content/drive/MyDrive/datasets /content/vldb2021_fauce/queries_featurization/build_graphs/dataset

In [ ]:
%%bash
source activate qf
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/title.csv --output output_matrix/matrix_title.csv
python rdc.py --input dataset/cast_info.csv --output output_matrix/matrix_cast_info.csv
python rdc.py --input dataset/movie_info.csv --output output_matrix/movie_info.csv
python rdc.py --input dataset/movie_companies.csv --output output_matrix/movie_companies.csv
python rdc.py --input dataset/movie_keyword.csv --output output_matrix/movie_keyword.csv
python rdc.py --input dataset/movie_info_idx.csv --output output_matrix/movie_info_idx.csv

In [ ]:
%%bash
source activate qf
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/title.csv --output output_matrix/matrix_title.csv

In [ ]:
%%bash
source activate qf
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/cast_info.csv --output output_matrix/matrix_cast_info.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_info.csv --output output_matrix/matrix_movie_info.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_companies.csv --output output_matrix/matrix_movie_companies.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_keyword.csv --output output_matrix/matrix_movie_keyword.csv

In [ ]:
%%bash
cd /content/vldb2021_fauce/queries_featurization/build_graphs
python rdc.py --input dataset/movie_info_idx.csv --output output_matrix/matrix_movie_info_idx.csv

In [ ]:
"""
http://papers.nips.cc/paper/5138-the-randomized-dependence-coefficient.pdf
"""
import argparse
import numpy as np
from scipy.stats import rankdata
import csv
import pandas

def rdc(x, y, f=np.sin, k=20, s=1/6., n=1):
    """
    Computes the Randomized Dependence Coefficient
    x,y: numpy arrays 1-D or 2-D
         If 1-D, size (samples,)
         If 2-D, size (samples, variables)
    f:   function to use for random projection
    k:   number of random projections to use
    s:   scale parameter
    n:   number of times to compute the RDC and
         return the median (for stability)

    According to the paper, the coefficient should be relatively insensitive to
    the settings of the f, k, and s parameters.
    """
    if n > 1:
        values = []
        for i in range(n):
            try:
                values.append(rdc(x, y, f, k, s, 1))
            except np.linalg.linalg.LinAlgError: pass
        return np.median(values)

    if len(x.shape) == 1: x = x.reshape((-1, 1))
    if len(y.shape) == 1: y = y.reshape((-1, 1))

    # Copula Transformation
    cx = np.column_stack([rankdata(xc, method='ordinal') for xc in x.T])/float(x.size)
    cy = np.column_stack([rankdata(yc, method='ordinal') for yc in y.T])/float(y.size)

    # Add a vector of ones so that w.x + b is just a dot product
    O = np.ones(cx.shape[0])
    X = np.column_stack([cx, O])
    Y = np.column_stack([cy, O])

    # Random linear projections
    Rx = (s/X.shape[1])*np.random.randn(X.shape[1], k)
    Ry = (s/Y.shape[1])*np.random.randn(Y.shape[1], k)
    X = np.dot(X, Rx)
    Y = np.dot(Y, Ry)

    # Apply non-linear function to random projections
    fX = f(X)
    fY = f(Y)

    # Compute full covariance matrix
    C = np.cov(np.hstack([fX, fY]).T)

    # Due to numerical issues, if k is too large,
    # then rank(fX) < k or rank(fY) < k, so we need
    # to find the largest k such that the eigenvalues
    # (canonical correlations) are real-valued
    k0 = k
    lb = 1
    ub = k
    while True:

        # Compute canonical correlations
        Cxx = C[:k, :k]
        Cyy = C[k0:k0+k, k0:k0+k]
        Cxy = C[:k, k0:k0+k]
        Cyx = C[k0:k0+k, :k]

        eigs = np.linalg.eigvals(np.dot(np.dot(np.linalg.pinv(Cxx), Cxy),
                                        np.dot(np.linalg.pinv(Cyy), Cyx)))

        # Binary search if k is too large
        if not (np.all(np.isreal(eigs)) and
                0 <= np.min(eigs) and
                np.max(eigs) <= 1):
            ub -= 1
            k = (ub + lb) // 2
            continue
        if lb == ub: break
        lb = k
        if ub == lb + 1:
            k = ub
        else:
            k = (ub + lb) // 2

    return np.sqrt(np.max(eigs))

In [ ]:
# import sys
# directory_path = '/content/vldb2021_fauce/queries_featurization/build_graphs'
# sys.path.append(directory_path)
# from rdc import rdc
df_title=pd.read_csv('/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/title.csv')
id=df_title['id'].to_numpy()
kind_id=df_title['kind_id'].to_numpy()
production_year=df_title['production_year'].to_numpy()
print(rdc(id, id))
print(rdc(id, kind_id))
print(rdc(id, production_year))
print(rdc(kind_id, id))
print(rdc(kind_id, kind_id))
print(rdc(kind_id, production_year))
print(rdc(production_year, id))
print(rdc(production_year, kind_id))
print(rdc(production_year, production_year))

0.9999997470941973
0.894738940780321
0.29930996483117445
0.8946344046558942
0.9999999996684112
0.358458071761189
0.2986015217150516
0.3584786096580924
0.999999999646699


In [ ]:
df_ci=pd.read_csv('/content/vldb2021_fauce/queries_featurization/build_graphs/dataset/cast_info.csv')
person_id=df_ci['person_id'].to_numpy()
movie_id=df_ci['movie_id'].to_numpy()
role_id=df_ci['role_id'].to_numpy()
print(rdc(person_id, person_id))
print(rdc(person_id, movie_id))
print(rdc(person_id, role_id))
print(rdc(movie_id, person_id))
print(rdc(movie_id, movie_id))
print(rdc(movie_id, role_id))
print(rdc(role_id, person_id))
print(rdc(role_id, movie_id))
print(rdc(role_id, role_id))

0.9999999866630367
0.03943053282076706


KeyboardInterrupt: 

In [ ]:
# df_title = df_title[['id', 'kind_id', 'production_year']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_title.csv
# title correlated columns: title.id-title.kind_id
# Local dependency graph:
# vertex: id, kind_id, production_year
# edge: id - kind_id

0.9999960805667225,0.8951884822208703,0.2992750478079193
0.8947404143347979,0.9999999998969117,0.35846943935982706
0.30004202069634534,0.3584044146009544,0.9999999992148845


In [ ]:
# df_ci = df_ci[['person_id', 'movie_id', 'role_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_cast_info.csv
# cast_info correlated columns: cast_info.person_id-cast_info.role_id
# Local dependency graph:
# vertex: person_id, movie_id, role_id
# edge: person_id - role_id

0.9999999969504254,0.03130249404100904,0.800494035287856
0.03164761790229394,0.9999999782046025,0.03099804764422859
0.8401848380486133,0.04396121941061839,0.9999999706808845


In [ ]:
# df_mi = df_mi[['movie_id', 'info_type_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_info.csv
# movie_info correlated columns: none
# Local dependency graph:
# vertex: movie_id, info_type_id
# edge: None

0.9999999967757665,0.29119507968304725
0.2911792952633776,0.9995811620149797


In [ ]:
# df_mc = df_mc[['movie_id','company_id','company_type_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_companies.csv
# Local dependency graph:
# vertex: movie_id, company_id, company_type_id
# edge: movie_id-company_id, company_type_id - movie_id, company_id-company_type_id

0.9999999973613892,0.40780435949547805,0.7608115012445112
0.40769984198946585,0.9999999251789405,0.6099328964952723
0.7608454747500846,0.6094852265463174,0.999992730538565


In [ ]:
# df_mk = df_mk[['movie_id', 'keyword_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_keyword.csv
# movie_keyword correlated columns: none

# Local dependency graph:
# vertex: movie_id, keyword_id
# edge: none

0.9999996571476205,0.07100057489162931
0.07101307648504732,0.9999989694599533


In [ ]:
# df_mi_idx = df_mi_idx[['movie_id', 'info_type_id']]
!cat /content/vldb2021_fauce/queries_featurization/build_graphs/output_matrix/matrix_movie_info_idx.csv
# movie_info_idx correlated columns: movie_info_idx.info_type_id-movie_info_idx.movie_id

# Local dependency graph:
# vertex: info_type_id, movie_id
# edge: info_type_id - movie_id

0.999999999787938,0.5624283217110245
0.5621264188269423,0.9999985194596575


- For each "Table" in the database, we calculate the Randomized Dependence Coefficient(RDC) values for each pair of "Columns" in each "Table" to build the local columns-dependency graph;
  - title: title.id-title.kind_id
  - cast_info: person_id - role_id
  - movie_info: none
  - movie_companies: movie_id-company_id, company_type_id-movie_id, company_id-company_type_id
  - movie_keyword: none
  - movie_info_idx: movie_info_idx.info_type_id-movie_info_idx.movie_id
```csv
title.id movie_id 1
kind_id 2
production_year 3
person_id 4
role_id 5
info_type_id 6
company_id 7
company_type_id 8
keyword_id 9
```

### Build a global columns-dependency graph for the database
Connect the local columns-dependency graph based on the PK and FK relationships among "Tables" to build the global columns-dependency graph;

In [ ]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_column_names.txt

title.id 1
title.kind_id 2
title.production_year 3
cast_info.person_id 4
cast_info.movie_id 1
cast_info.role_id 5
movie_info.movie_id 1
movie_info.info_type_id 6
movie_companies.movie_id 1
movie_companies.company_id 7
movie_companies.company_type_id 8
movie_keyword.movie_id 1
movie_keyword.keyword_id 9
movie_info_idx.movie_id 1
movie_info_idx.info_type_id 6

In [ ]:
!cat /content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_columns_dependency_graph.edgelist

1 2
3 3
4 5
1 6
1 7
1 8
7 8
9 9

### Use graph embedding for encoding
Given the global columns-dependency graph as the input, use the same graph embedding method (the method to represent "Tables" as vectors) to get the embeddings for the "Columns" in the database.

In [ ]:
# C = 9
%%bash
cd /content/vldb2021_fauce/queries_featurization/graph_embedding
source activate qf
python main.py --input graph/global_columns_dependency_graph.edgelist --output emb/global_columns.emd --dimensions 9

Walk iteration:
1 / 3
2 / 3
3 / 3


/usr/local/envs/qf/lib/python2.7/site-packages/gensim/models/word2vec.py:651: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


## Combine all vectors
| Type    | Table       | Join          | Column          | Predicate   |
|---------|-------------|---------------|-----------------|-------------|
| Segment | ⟨Tables⟩    | ⟨Joins⟩       | ⟨Columns⟩       | ⟨Values⟩    |
| Method  | Embedding   | Joins2Vec     | Columns2Vec     | Range       |
| Seg. Size | m⌈log(m+1)⌉ | n           | C               | 2 × C       |

- m: the number of tables in the database
  - JOB-light: 6
- n: the number of possible join relationships among the database tables
  - JOB-light: 5
- C: the total number of different columns in the database
  - JOB-light: 9
- the feature vector has a length of $L = m\lceil log(m+1)\rceil+n+3C=6*3+5+9*3=50$

### Tables

```csv
title 1
cast_info 2
movie_info 3
movie_companies 4
movie_keyword 5
movie_info_idx 6
```

In [ ]:
import numpy as np
def vectorize_tables(tables, f_emb='/content/vldb2021_fauce/queries_featurization/graph_embedding/emb/graph_node.emd',
                     f_table_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/table_names.txt'):
  """convert tables into a single vector
    Args:
      tables: string, Comma-separated table names
      f_emb: generated embedding file, its first line is "<number of tables> <number of embedding size>", e.g.: 6 3
        other lines are: <table_id> <embedding_feature_1> <embedding_feature_2> <embedding_feature_3> ...
      f_table_names: consists of <number of tables> lines, each line is: <table_name> <table_id>  e.g.: title 1

    Returns:
      Vectorization of input tables, its feature order is sorted by <table_id>. Numpy arry, Vector size is (1, <number of tables> * <number of embedding size>)
      All tables that are not included in tables, the vector is filled with 0.
    E.g.: input: tables = "cast_info,title",
                content in f_emb = "
                            3 2
                            2 0.12 0.1
                            1 0.11 0.3
                            3 0.5 0.7
                          "
                content in f_table_names =
                "
                    title 1
                    cast_info 2
                    movie_info 3
                "
          output: [0.11, 0.3, 0.12, 0.1, 0, 0]
    """
    # Read embedding file
  with open(f_emb, 'r') as f:
    num_tables, embedding_size = map(int, f.readline().strip().split())
    embeddings = {}
    for line in f:
        parts = line.strip().split()
        table_id = int(parts[0])
        embedding = list(map(float, parts[1:]))
        embeddings[table_id] = embedding

    # Read table names file
  with open(f_table_names, 'r') as f:
    table_id_map = {}
    for line in f:
        table_name, table_id = line.strip().split()
        table_id_map[table_name] = int(table_id)
    # Process input tables
  input_tables = tables.split(',')
  # Create the output vector
  output_vector = np.zeros((1, num_tables * embedding_size))

  # Fill the output vector with embeddings for input tables
  for table in input_tables:
      if table in table_id_map:
          table_id = table_id_map[table]
          if table_id in embeddings:
              start_idx = (table_id - 1) * embedding_size
              output_vector[0, start_idx:start_idx+embedding_size] = embeddings[table_id]

  return output_vector

In [ ]:
res = vectorize_tables("movie_companies,movie_info_idx,title")
print(res)
print(res.shape)


[[-0.18052  -0.171498  0.166586  0.        0.        0.        0.
   0.        0.       -0.002204  0.161061  0.000822  0.        0.
   0.        0.049148 -0.038398  0.034771]]
(1, 18)


### Joins

In [ ]:
import numpy as np
def vectorize_joins(joins, f_emb='/content/vldb2021_fauce/queries_featurization/Joins2Vec/embeddings/node_edges_dims_5_epochs_3_embeddings.txt',
                     f_table_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/table_names.txt'):
  """convert tables into a single vector
    Args:
      joins: string, Comma-separated join names, e.g., "title.id=cast_info.movie_id,title.id=movie_companies.movie_id"
      f_emb: generated embedding file, its first line is "<number_of_join_relationships> <number_of_embedding_size>", e.g.: 19 30
        other lines are: <join_relationship> <embedding_feature_1> <embedding_feature_2> <embedding_feature_3> ...
        <join_relationship> is in form of <table_id>#<title_id>#...#<table_id>.
        We only consider 1-degree join, e.g., 1#2
      f_table_names: consists of <number of tables> lines, each line is: <table_name> <table_id>  e.g.: title 1

    Returns:
      Vectorization of input joins, summing up embeddings of existing join relationships in Joins. Numpy arry, Vector size is (1, <number of embedding size>)

    E.g.: input: joins = "title.id=movie_info.movie_id,title.id=cast_info.movie_id,cast_info.movie_id=movie_info.movie_id",
                content in f_emb = "
                            3 2
                            2#1 0.12 0.1
                            3#1 0.11 0.3
                            2#1##1#3 0.5 0.7
                            UNK -1.0 -1.0
                          "
                content in f_table_names =
                "
                    title 1
                    cast_info 2
                    movie_info 3
                "
          joins can be converted into: "1#3,1#2,2#3", which is equivalent to "3#1,2#1,UNK", because 2#3 does not exist in f_emb
          so we need to sum up the embedding of 3#1, 2#1 and UNK i.e., [0.11 0.3] + [0.12 0.1] +[-1.0,-1.0].
          output: [-0.77,-0.6]
    """
  # Read embedding file
  with open(f_emb, 'r') as f:
      num_join_relationships, embedding_size = map(int, f.readline().strip().split())
      embeddings = {}
      for line in f:
          parts = line.strip().split()
          join_relationship = parts[0]
          embedding = np.array(list(map(float, parts[1:])))
          embeddings[join_relationship] = embedding
  # Read table names file
  with open(f_table_names, 'r') as f:
      table_id_map = {line.strip().split()[0]: int(line.strip().split()[1]) for line in f}
      id_table_map = {v: k for k, v in table_id_map.items()}
  output_vector = np.zeros(embedding_size)
  if len(joins) == 0:
    output_vector += embeddings['UNK']
    return output_vector.reshape(1, -1)
  # Process input joins
  input_joins = joins.split(',')

  # Convert join names to table IDs
  converted_joins = []
  for join in input_joins:
      table1, table2 = join.split('=')
      table1_name, _ = table1.split('.')
      table2_name, _ = table2.split('.')
      id1, id2 = table_id_map[table1_name], table_id_map[table2_name]
      converted_joins.append(f"{id1}#{id2}")
  # Sum up the embeddings
  for join in converted_joins:
      if join in embeddings:
        output_vector += embeddings[join]
      # Check for reverse join (e.g., "2#1" instead of "1#2")
      elif '#'.join(reversed(join.split('#'))) in embeddings:
        output_vector += embeddings['#'.join(reversed(join.split('#')))]
      else:
        # Handle unknown join relationships
        output_vector += embeddings['UNK']
  return output_vector.reshape(1, -1)

In [ ]:
joins = "title.id=movie_companies.movie_id,movie_companies.movie_id=movie_info_idx.movie_id"
res = vectorize_joins(joins)
print(res)
print(res.shape)

[[ 1.54441576 -2.5088322   0.91622484  0.162513    1.45248982]]
(1, 5)


In [ ]:
joins = ""
res = vectorize_joins(joins)
print(res)
print(res.shape)

[[ 0.57381356 -1.227445    0.16418804 -1.1904593   1.3114746 ]]
(1, 5)


### Columns

In [ ]:
import pandas as pd
import os
def get_range(dataset_path="/content/drive/MyDrive/datasets", output="/content/vldb2021_fauce/queries_featurization/featurization/col_values_range.csv"):
  """summarize the range (min value & max value) of values for each column, and save the result to output
    Args:
      dataset_path: string, folder that contains csv files
      output: string, output file path
    Returns:
      df_output, pandas dataframe, the first line is header, other lines are in the
        form of <file_name.column_name> <min value> <max value>, e.g.,
        column min max
        title.id 1 2528298
        title.kind_id 1 7
        title.production_year 1880 2019
        cast_info.movie_id 1 2525975
        cast_info.role_id 1 11
  """
  # Create an empty list to store the results
  result_list = []

  # Loop through all CSV files in the dataset_path directory
  for filename in os.listdir(dataset_path):
      if filename.endswith(".csv"):
          # Load the CSV file into a Pandas DataFrame
          df = pd.read_csv(os.path.join(dataset_path, filename))

          # Loop through each column in the DataFrame
          for col in df.columns:
              # Calculate the min and max values for the column
              col_min = df[col].min()
              col_max = df[col].max()

              # Add the results to the list
              result_list.append({'column': f"{filename.split('.')[0]}.{col}", 'min': col_min, 'max': col_max})
  # Create the output DataFrame from the list
  df_output = pd.DataFrame(result_list)
  # Save the output DataFrame to a CSV file
  df_output.to_csv(output, index=False)
  return df_output

In [ ]:
# get_range()

,column,min,max
0,title.id,1.0,2528298.0
1,title.kind_id,1.0,7.0
2,title.production_year,1880.0,2019.0
3,cast_info.person_id,1.0,4061926.0
4,cast_info.movie_id,1.0,2525975.0
5,cast_info.role_id,1.0,11.0
6,movie_info.movie_id,1.0,2526430.0
7,movie_info.info_type_id,1.0,110.0
8,movie_companies.movie_id,2.0,2525745.0
9,movie_companies.company_id,1.0,234997.0


In [ ]:
import numpy as np
import pandas as pd
import os
def extract_columns_and_values(where_clause, f_values_range="/content/vldb2021_fauce/queries_featurization/featurization/col_values_range.csv"):
  """extract columns and range of values from where clause
    Read pandas dataframe from f_values_range. Edit the dataframe according to where_clause
    and return the updated df.
    Args:
      where_clause: string, Comma-separated conjunctive predicates on the columns, e.g., "movie_info.info_type_id,=,101,movie_info.info_type_id,=,3,title.production_year,>,2005,title.production_year,<,2008"
      f_values_range: default values_range.csv, containing <file_name.column_name> <min value> <max value>, e.g.,
        column min max
        title.id 1.0 2528298.0
    Returns:
      pandas dataframe containing updated values_range, headers are ['column', 'min', "max"]. data are in the
        form of <file_name.column_name> <min value> <max value>, e.g.,
        column min max
        title.id 1 10
    e.g.,
      input:
        where_clause = "movie_info.info_type_id,=,101,title.production_year,>,2005,title.production_year,<,2008"
        f_values_range="/content/vldb2021_fauce/queries_featurization/featurization/values_range.csv"
        file content in f_values_range:
          column min max
          movie_info.info_type_id 1 110
          title.production_year 1880 2019
      output:
        column min max
        movie_info.info_type_id 101 101
        title.production_year 2005 2008
  """
  # Read the values_range CSV file into a Pandas DataFrame
  df_values_range = pd.read_csv(f_values_range)
  if len(where_clause)==0:
    return df_values_range
  # Split the where_clause into individual predicates
  predicates = where_clause.split(',')

  # Initialize an empty list to store the updated values range
  # Initialize an empty Dictionary to store the updated values range
  result_dict = {}
  for i in range(0, len(predicates), 3):
    column_name = predicates[i]
    operator = predicates[i+1]
    value = float(predicates[i+2])
    # Find the corresponding row in the values_range DataFrame
    row = df_values_range[df_values_range['column'] == column_name].iloc[0]

    # Update the min and max values based on the operator
    if operator == '=':
        result_dict[column_name] = {'min': value, 'max': value}
    elif operator == '>' or operator == '>=':
      if column_name in result_dict:
        result_dict[column_name]['min'] = max(result_dict[column_name]['min'], value)
      else:
        result_dict[column_name] = {'min': value, 'max': row['max']}
    elif operator == '<' or operator == '<=':
      if column_name in result_dict:
        result_dict[column_name]['max'] = min(result_dict[column_name]['max'], value)
      else:
        result_dict[column_name] = {'min': row['min'], 'max': value}
    else:
        raise ValueError(f"Invalid operator: {operator}")
  df_output = pd.DataFrame.from_dict(result_dict, orient='index').reset_index()
  df_output.columns = ['column', 'min', 'max']
  return df_output

In [ ]:
df = extract_columns_and_values("movie_info.info_type_id,=,101,title.production_year,>,2005,title.production_year,<,2008")
df

,column,min,max
0,movie_info.info_type_id,101.0,101.0
1,title.production_year,2005.0,2008.0


In [ ]:
df['column'].to_list()

['movie_info.info_type_id', 'title.production_year']

In [ ]:
extract_columns_and_values("")

,column,min,max
0,title.id,1.0,2528298.0
1,title.kind_id,1.0,7.0
2,title.production_year,1880.0,2019.0
3,cast_info.person_id,1.0,4061926.0
4,cast_info.movie_id,1.0,2525975.0
5,cast_info.role_id,1.0,11.0
6,movie_info.movie_id,1.0,2526430.0
7,movie_info.info_type_id,1.0,110.0
8,movie_companies.movie_id,2.0,2525745.0
9,movie_companies.company_id,1.0,234997.0


In [ ]:
import numpy as np
def vectorize_columns(columns, f_emb='/content/vldb2021_fauce/queries_featurization/graph_embedding/emb/global_columns.emd',
                     f_col_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_column_names.txt'):
  """convert tables into a single vector
    Args:
      columns: list, columns names in the form of <table_name>.<column_name>, e.g., ["title.id","cast_info.movie_id","movie_companies.movie_id"]
      f_emb: generated embedding file, its first line is "<number_of_column> <number_of_embedding_size>", e.g.: 19 30
        other lines are: <column_id> <embedding_feature_1> <embedding_feature_2> <embedding_feature_3> ...
        If a column_id appears multiple times in columns, we only consider the first occurrence.
      f_col_names: each line is: <column_name> <column_id>, e.g.: title.id 1.
        some columns may have same column_id with different column_name.

    Returns:
      Vectorization of input joins, summing up embeddings of existing unique columns in columns.
        Numpy arry, Vector size is (1, <number of embedding size>)

    E.g.: input: columns = ["title.id","cast_info.movie_id","movie_companies.company_type_id"],
                content in f_emb = "
                            3 2
                            1 0.1 0.1
                            2 0.2 0.2
                            6 0.3 0.3
                          "
                content in f_col_names =
                "
                    title.id 1
                    cast_info.movie_id 1
                    movie_companies.company_type_id 6
          columns can be converted into a set: (1, 6)
          so we need to sum up the embedding of 1 and 6 i.e., [0.1, 0.1] + [0.3, 0.3]
          output: [0.4,0.4]
    """
  # Read the embedding file
  with open(f_emb, 'r') as f:
      num_columns, embedding_size = map(int, f.readline().strip().split())
      embeddings = {}
      for line in f:
          parts = line.strip().split()
          column_id = int(parts[0])
          embedding = np.array(list(map(float, parts[1:])))
          embeddings[column_id] = embedding

  # Read the column names file
  with open(f_col_names, 'r') as f:
      column_id_map = {line.strip().split()[0]: int(line.strip().split()[1]) for line in f}
  # Process the input columns
  column_ids = set()
  for column in columns:
    column_ids.add(column_id_map[column])
  # Compute the output vector
  output_vector = np.zeros(embedding_size)
  for column_id in column_ids:
      if column_id in embeddings:
          output_vector += embeddings[column_id]
  return output_vector.reshape(1, -1)

In [ ]:
df = extract_columns_and_values("title.id,=,1,movie_info.movie_id,<,10,movie_info.info_type_id,=,101,title.production_year,>,2005,title.production_year,<,2008")
columns = df['column'].to_list()
vector_columns = vectorize_columns(columns)
print(vector_columns)
print(vector_columns.shape)

[[-0.095601 -0.060428  0.128408  0.022814  0.04561  -0.038919  0.053163
   0.070842 -0.025461]]
(1, 9)


In [ ]:
df = extract_columns_and_values("")
columns = df['column'].to_list()
vector_columns = vectorize_columns(columns)
print(vector_columns)
print(vector_columns.shape)

[[ 0.002907 -0.103374 -0.007038  0.016386  0.02425  -0.031825  0.104506
   0.168587  0.094116]]
(1, 9)


### Values

In [ ]:
import numpy as np
import pandas as pd
import os
def vectorize_values(df_values_range, f_default_ranges='/content/vldb2021_fauce/queries_featurization/featurization/default_value.csv',
                     f_col_ranges='/content/vldb2021_fauce/queries_featurization/featurization/col_values_range.csv',
                     f_col_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_column_names.txt'):
  """convert tables into a single vector
    Args:
      df_values_range: panda data frame, its headers are ['column', 'min', 'max'].
        <column> is in the form of <table_name>.<column_name>, e.g., "title.id", 1.0, 100.0
      f_default_ranges: string, file containing default ranges for each column_id
      f_col_names: each line is: <column_name> <column_id>, e.g.: title.id 1.
        some columns may have same column_id with different column_name.

    Returns:
      Vectorization of all columns' values range, its feature order is sorted by <column_id>.
        Each column_id has two featurese, the first is min value, the second is max value.
        If any column doesn't appear in df_values_range, the feature is filled with the default value in f_default_ranges.
        <number of embedding size> is the number of unique <column_id> in f_col_names
        Numpy arry, Vector size is (1, <number of embedding size>)
    P.S.:
      Flow:
        0. Load default ranges in f_default_ranges.
      For each values range: 1. extract the column_name from column, get its column_id from f_col_names
        2. find the corresponding row in f_col_ranges
        3. A column's min value is max(df_values_range['column'==column_name]['min'], f_col_ranges['column'==column_name]['min'], f_default_ranges['column_id'==column_id]['min'])
          A column's max value is min(df_values_range['column'==column_name]['max'], f_col_ranges['column'==column_name]['max'], f_default_ranges['column_id'==column_id]['max'])
    E.g.: input: df_values_range =
                    column,min,max
                    movie_info.movie_id,2.0,2525975.0
                    movie_info.info_type_id,101.0,101.0
                    title.production_year,2005.0,2008.0,
                content in f_default_ranges =
                            column_id,min,max
                            1,1.0,2528298.0
                            2,1.0,7.0
                            3,1880.0,2019.0
                            5,1.0,113.0

                content in f_col_ranges =
                            column,min,max
                            movie_info.info_type_id,1,101.0
                            title.production_year,1880.0,2019.0,
                            title.id,1.0,2528298.0
                            title.kind_id,1.0,7.0
                            movie_info.movie_id,1.0,2525975.0
                content in f_col_names =
                    title.id 1
                    title.kind_id 2
                    title.production_year 3
                    movie_info.info_type_id 5
                    movie_info.movie_id 1
          columns can be converted into a set: (1, 6)
          there're 4 different column_id in f_col_names, so the output shape is (1, 8)
          output: [2.0,2525975.0,1.0,7.0,2005.0,2008.0,101.0,101.0]
    """
    # Load default ranges
  df_default_ranges = pd.read_csv(f_default_ranges)
  # Load column ranges
  df_col_ranges = pd.read_csv(f_col_ranges)

  # Load column names and IDs
  col_id_map = {}
  with open(f_col_names, 'r') as f:
      for line in f:
          col_name, col_id = line.strip().split()
          col_id_map[col_name] = int(col_id)

  # Get unique column IDs
  unique_col_ids = sorted(set(col_id_map.values()))

  # Initialize output vector
  output_vector = np.zeros(len(unique_col_ids) * 2)

  # Process each column
  for col_name in col_id_map.keys():
      col_id = col_id_map[col_name]

      # Get default range
      default_min = df_default_ranges[df_default_ranges['column_id'] == col_id]['min'].values[0]
      default_max = df_default_ranges[df_default_ranges['column_id'] == col_id]['max'].values[0]

      # Get column range
      col_min = df_col_ranges[df_col_ranges['column'] == col_name]['min'].values[0] if col_name in df_col_ranges['column'].values else default_min
      col_max = df_col_ranges[df_col_ranges['column'] == col_name]['max'].values[0] if col_name in df_col_ranges['column'].values else default_max

      # Get input range
      input_min = df_values_range[df_values_range['column'] == col_name]['min'].values[0] if col_name in df_values_range['column'].values else default_min
      input_max = df_values_range[df_values_range['column'] == col_name]['max'].values[0] if col_name in df_values_range['column'].values else default_max

      # Calculate final min and max
      final_min = max(input_min, col_min, default_min)
      final_max = min(input_max, col_max, default_max)

      # Set values in output vector
      index = unique_col_ids.index(col_id)
      output_vector[index*2] = final_min
      output_vector[index*2 + 1] = final_max
  return output_vector.reshape(1, -1)

In [ ]:
df_values_range = extract_columns_and_values("title.id,=,1,movie_info.movie_id,<,10,movie_info.info_type_id,=,101,title.production_year,>,2005,title.production_year,<,2008")
print(df_values_range)
values = vectorize_values(df_values_range)
print(values)
print(values.shape)

                    column     min     max
0                 title.id     1.0     1.0
1      movie_info.movie_id     1.0    10.0
2  movie_info.info_type_id   101.0   101.0
3    title.production_year  2005.0  2008.0
[[2.000000e+00 2.525793e+06 1.000000e+00 7.000000e+00 2.005000e+03
  2.008000e+03 1.000000e+00 4.061926e+06 1.000000e+00 1.100000e+01
  9.900000e+01 1.130000e+02 1.000000e+00 2.349970e+05 1.000000e+00
  2.000000e+00 1.000000e+00 1.341700e+05]]
(1, 18)


In [ ]:
df_values_range = extract_columns_and_values("")
print(df_values_range)
values = vectorize_values(df_values_range)
print(values)
print(values.shape)

                             column     min        max
0                          title.id     1.0  2528298.0
1                     title.kind_id     1.0        7.0
2             title.production_year  1880.0     2019.0
3               cast_info.person_id     1.0  4061926.0
4                cast_info.movie_id     1.0  2525975.0
5                 cast_info.role_id     1.0       11.0
6               movie_info.movie_id     1.0  2526430.0
7           movie_info.info_type_id     1.0      110.0
8          movie_companies.movie_id     2.0  2525745.0
9        movie_companies.company_id     1.0   234997.0
10  movie_companies.company_type_id     1.0        2.0
11           movie_keyword.movie_id     2.0  2525971.0
12         movie_keyword.keyword_id     1.0   134170.0
13          movie_info_idx.movie_id     2.0  2525793.0
14      movie_info_idx.info_type_id    99.0      113.0
[[2.000000e+00 2.525793e+06 1.000000e+00 7.000000e+00 1.880000e+03
  2.019000e+03 1.000000e+00 4.061926e+06 1.000000e+00

## Combine all vectors into training data

In [ ]:
def process_row(row,
                f_table_emb='/content/vldb2021_fauce/queries_featurization/graph_embedding/emb/graph_node.emd',
                f_table_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/table_names.txt',
                f_join_emb='/content/vldb2021_fauce/queries_featurization/Joins2Vec/embeddings/node_edges_dims_5_epochs_3_embeddings.txt',
                f_values_range="/content/vldb2021_fauce/queries_featurization/featurization/col_values_range.csv",
                f_col_emb='/content/vldb2021_fauce/queries_featurization/graph_embedding/emb/global_columns.emd',
                f_col_names='/content/vldb2021_fauce/queries_featurization/graph_embedding/graph/global_column_names.txt',
                f_default_ranges='/content/vldb2021_fauce/queries_featurization/featurization/default_value.csv',
                ):
  """
    Args:
      f_table_emb: generated embedding file, default=
      f_table_names: consists of <number of tables> lines, each line is: <table_name> <table_id>  e.g.: title 1
      f_join_emb: generated embedding file, We only consider 1-degree join, e.g., 1#2
      f_values_range: values bounder of each column, (statistics)
      f_col_emb: generated embedding file for columns
      f_col_names: each line is: <column_name> <column_id>, e.g.: title.id 1.
        some columns may have same column_id with different column_name
      f_default_ranges: default range for each column_id
    Returns:
  """
  arr_t = vectorize_tables(row['Tables'], f_emb=f_table_emb, f_table_names=f_table_names)
  arr_j = vectorize_joins(row['Joins'], f_emb=f_join_emb, f_table_names=f_table_names)
  df_values_range = extract_columns_and_values(row['Filters'], f_values_range=f_values_range)
  arr_c = vectorize_columns(df_values_range['column'].tolist(), f_emb=f_col_emb, f_col_names=f_col_names)
  arr_v = vectorize_values(df_values_range, f_default_ranges=f_default_ranges,
                           f_col_ranges=f_values_range, f_col_names=f_col_names)
  row_df = pd.DataFrame({
      **{f't_{i}': arr_t[0, i] for i in range(arr_t.shape[1])},
      **{f'j_{i}': arr_j[0, i] for i in range(arr_j.shape[1])},
      **{f'c_{i}': arr_c[0, i] for i in range(arr_c.shape[1])},
      **{f'v_{i}': arr_v[0, i] for i in range(arr_v.shape[1])},
      'Cardinality': row['Cardinality']
  }, index=[0])
  return row_df

In [ ]:
def get_vectorization(f_input):
  df = pd.read_csv(f_input, sep='#', header=None, names=['Tables', 'Joins', 'Filters', 'Cardinality'])
  df['Joins']=df['Joins'].fillna('')
  df['Filters']=df['Filters'].fillna('')
  df_list = df.apply(process_row, axis=1).tolist()
  new_df = pd.concat(df_list, ignore_index=True)
  return new_df

In [ ]:
import time
import pandas as pd

f_input="/content/vldb2021_fauce/training_data/raw_data/JobLight-train.csv"

df = pd.read_csv(f_input, sep='#', header=None, names=['Tables', 'Joins', 'Filters', 'Cardinality'])
df['Joins']=df['Joins'].fillna('')
df['Filters']=df['Filters'].fillna('')

start_time = time.time()
new_df = get_vectorization(f_input)
execution_time = time.time() - start_time
print(f"Featurization Execution time for {len(df)} rows: {execution_time} seconds. Average time: {execution_time*1000/len(df)} ms")

Featurization Execution time for 100000 rows: 3381.091047525406 seconds. Average time: 33.81091047525406 ms


```
Featurization Execution time for 100000 rows: 3381.091047525406 seconds.
Average time: 33.81091047525406 ms
```

In [ ]:
new_df.to_csv("/content/vldb2021_fauce/training_data/featurization/JobLight-train.csv", index=False)

In [ ]:
f_input="/content/vldb2021_fauce/training_data/raw_data/job-light.csv"
new_df = get_vectorization(f_input)
new_df.to_csv("/content/vldb2021_fauce/training_data/featurization/job-light.csv", index=False)

In [ ]:
import time
import pandas as pd

f_input="/content/vldb2021_fauce/training_data/raw_data/job-light-content-2500.csv"

df = pd.read_csv(f_input, sep='#', header=None, names=['Tables', 'Joins', 'Filters', 'Cardinality'])
df['Joins']=df['Joins'].fillna('')
df['Filters']=df['Filters'].fillna('')

start_time = time.time()
new_df = get_vectorization(f_input)
execution_time = time.time() - start_time
print(f"Featurization Execution time for {len(df)} rows: {execution_time} seconds. Average time: {execution_time*1000/len(df)} ms")

Featurization Execution time for 2070 rows: 78.17388105392456 seconds. Average time: 37.76515992943215 ms


f_input="training_data/raw_data/job-light-content-2500.csv"
```
Featurization Execution time for 2070 rows: 78.17388105392456 seconds. Average time: 37.76515992943215 ms
```

In [ ]:
new_df.to_csv("/content/vldb2021_fauce/training_data/featurization/job-light-content-2500.csv", index=False)

# Training

## Training Data pre-processing
Idealy, raw_data is in the following form without alias.
```
movie_companies,movie_info_idx,title#title.id=movie_companies.movie_id,movie_companies.movie_id=movie_info_idx.movie_id#movie_info_idx.info_type_id,=,101.0,title.kind_id,=,1.0,title.production_year,>=,2009.0,movie_companies.company_type_id,=,1.0#66264
```
Some datasets, e.g. job-light, have alias, we need pre-processing to generate queries without alias.
```
movie_companies mc,title t,movie_info_idx mi_idx#t.id=mc.movie_id,t.id=mi_idx.movie_id#mi_idx.info_type_id,=,112,mc.company_type_id,=,2#715
```

In [ ]:
import pandas as pd
def remove_alias(f_csv, sep='#', header=None, names=['Tables', 'Joins', 'Filters', 'Cardinality']):
  """remove alias in qeuries
  Input:
    f_csv: csv file containing queries with alias
      each query is in the form of:
        table_1 alias_1,table_2 alias_2,...,table_n alias_n#alias_1.column_name_1=alias_2.column_name_2,...,alias_m.column_name_m#alias_1.column_name_1,operator_1,value_1,alias_2.column_name_2,operator_2,value_2,...#cardinality
        e.g., title t,movie_info_idx mi_idx#t.id=mi_idx.movie_id#t.kind_id,=,7,mi_idx.info_type_id,>,99#283812
    sep: default='#',
    header: default=None,
    names: default=['Tables', 'Joins', 'Filters', 'Cardinality'], col names
  Returns:
    pandas dataframe without alias. Remove the alias in the first col, and replace the alias with full table name in other cols.
    e.g.,
      Input:
        title t,movie_info_idx mi_idx#t.id=mi_idx.movie_id#t.kind_id,=,7,mi_idx.info_type_id,>,99#283812
      Output:
        title,movie_info_idx#title.id=movie_info_idx.movie_id,t.kind_id,=,7,movie_info_idx.info_type_id,>,99#283812
  Example usage:
    df_without_alias = remove_alias('your_input_file.csv')
    df_without_alias.to_csv('your_output_file.csv', sep='#', header=False, index=False)
  """
  # Read the CSV file
  df = pd.read_csv(f_csv, sep=sep, header=header, names=names)

  def process_row(row):
      # Process Tables column
      tables = row['Tables'].split(',')
      table_dict = {}
      new_tables = []
      for table in tables:
          parts = table.strip().split()
          if len(parts) == 2:
              table_name, alias = parts
              table_dict[alias] = table_name
              new_tables.append(table_name)
          else:
              new_tables.append(parts[0])

      # Update Tables column
      row['Tables'] = ','.join(new_tables)

      # Process Joins column
      if pd.notna(row['Joins']):
          joins = row['Joins'].split(',')
          new_joins = []
          for join in joins:
              for alias, table in table_dict.items():
                  join = join.replace(f"{alias}.", f"{table}.")
              new_joins.append(join)
          row['Joins'] = ','.join(new_joins)
      else:
          row['Joins'] = ''  # or keep it as np.nan if you prefer

      # Process Filters column
      if pd.notna(row['Filters']):
          filters = row['Filters'].split(',')
          new_filters = []
          for i, item in enumerate(filters):
              if i % 3 == 0:  # This is a column name
                  for alias, table in table_dict.items():
                      item = item.replace(f"{alias}.", f"{table}.")
              new_filters.append(item)
          row['Filters'] = ','.join(new_filters)
      else:
          row['Filters'] = ''  # or keep it as np.nan if you prefer

      return row
  # Apply the processing to each row
  df = df.apply(process_row, axis=1)
  return df

In [ ]:
df_job_light = remove_alias("/content/vldb2021_fauce/training_data/raw_data/with_alias/job-light.csv")
df_job_light.to_csv("/content/vldb2021_fauce/training_data/raw_data/job-light.csv", sep='#', header=False, index=False)
df_job_light

,Tables,Joins,Filters,Cardinality
0,"movie_companies,title,movie_info_idx","title.id=movie_companies.movie_id,title.id=mov...","movie_info_idx.info_type_id,=,112,movie_compan...",715
1,"movie_companies,title,movie_info_idx","title.id=movie_companies.movie_id,title.id=mov...","movie_info_idx.info_type_id,=,113,movie_compan...",9
2,"movie_companies,title,movie_info_idx","title.id=movie_companies.movie_id,title.id=mov...","movie_info_idx.info_type_id,=,112,movie_compan...",47
3,"movie_companies,title,movie_info_idx","title.id=movie_companies.movie_id,title.id=mov...","movie_info_idx.info_type_id,=,113,movie_compan...",16
4,"movie_companies,title,movie_keyword","title.id=movie_companies.movie_id,title.id=mov...","movie_keyword.keyword_id,=,117",148552
...,...,...,...,...
65,"title,movie_info,movie_info_idx,movie_keyword,...","title.id=movie_info.movie_id,title.id=movie_ke...","title.production_year,>,2000,movie_info.info_t...",26718423
66,"title,movie_info,movie_info_idx,movie_keyword,...","title.id=movie_info.movie_id,title.id=movie_ke...","title.production_year,>,2005,movie_info.info_t...",15813608
67,"title,movie_info,movie_companies,cast_info,mov...","title.id=movie_info.movie_id,title.id=movie_co...","cast_info.role_id,=,2,movie_info.info_type_id,...",1067496
68,"title,movie_info,movie_companies,cast_info,mov...","title.id=movie_info.movie_id,title.id=movie_co...","cast_info.role_id,=,2,movie_info.info_type_id,...",268172


In [ ]:
df_job_light_train = remove_alias("/content/vldb2021_fauce/training_data/raw_data/with_alias/JobLight-train.csv")
df_job_light_train['Joins']=df_job_light_train['Joins'].fillna('')
df_job_light_train['Filters']=df_job_light_train['Filters'].fillna('')
df_job_light_train.to_csv("/content/vldb2021_fauce/training_data/raw_data/JobLight-train.csv", sep='#', header=False, index=False)


## Normalization & Log transformation
- According to the paper, we must apply a log transformation for labels (cardinality).




In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
def normalize_features(df):
  features = df.iloc[:, :-1]
  labels = df.iloc[:, -1]
  # normalize features
  scaler = MinMaxScaler()
  normalized_features = scaler.fit_transform(features)
  normalized_features_df = pd.DataFrame(normalized_features, columns=features.columns)
  df_normalized = pd.concat([normalized_features_df, labels], axis=1)
  return df_normalized

In [ ]:
def log_labels(df):
  features = df.iloc[:, :-1]
  labels = df.iloc[:, -1]
  # log 2 cardinality
  labels_adjusted = np.where(labels < 1, 1, labels)
  labels_transformed = np.log2(labels_adjusted)
  labels_df = pd.DataFrame(labels_transformed, columns=['cardinality'])
  return pd.concat([features, labels_df], axis=1)

In [ ]:
def transform_features_and_labels(f_input, f_output, normalization=False):
  df = pd.read_csv(f_input)
  if normalization:
    df = normalize_features(df)
  df = log_labels(df)
  df.to_csv(f_output, index=False)
  return df

In [ ]:
f_input='/content/vldb2021_fauce/training_data/featurization/job-light.csv'
f_output='/content/vldb2021_fauce/models_training/datasets/job-light.csv'
transform_features_and_labels(f_input, f_output, normalization=False)

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,cardinality
0,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002204,...,11.0,112.0,112.0,1.0,234997.0,2.0,2.0,1.0,134170.0,9.481799
1,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002204,...,11.0,113.0,113.0,1.0,234997.0,2.0,2.0,1.0,134170.0,3.169925
2,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002204,...,11.0,112.0,112.0,1.0,234997.0,2.0,2.0,1.0,134170.0,5.554589
3,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002204,...,11.0,113.0,113.0,1.0,234997.0,2.0,2.0,1.0,134170.0,4.000000
4,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,117.0,117.0,17.180609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,-0.142707,0.037522,0.148151,-0.002204,...,11.0,101.0,101.0,1.0,234997.0,1.0,2.0,1.0,134170.0,24.671332
66,-0.18052,-0.171498,0.166586,0.000000,0.000000,0.000000,-0.142707,0.037522,0.148151,-0.002204,...,11.0,101.0,101.0,1.0,234997.0,1.0,2.0,1.0,134170.0,23.914663
67,-0.18052,-0.171498,0.166586,0.134461,-0.017551,-0.034228,-0.142707,0.037522,0.148151,-0.002204,...,2.0,99.0,113.0,1.0,234997.0,1.0,2.0,7084.0,7084.0,20.025799
68,-0.18052,-0.171498,0.166586,0.134461,-0.017551,-0.034228,-0.142707,0.037522,0.148151,-0.002204,...,2.0,99.0,113.0,1.0,234997.0,1.0,2.0,7084.0,7084.0,18.032799


In [ ]:
f_input='/content/vldb2021_fauce/training_data/featurization/JobLight-train.csv'
f_output='/content/vldb2021_fauce/models_training/datasets/JobLight-train.csv'
transform_features_and_labels(f_input, f_output, normalization=False)

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,cardinality
0,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1.0,134170.0,18.114576
1,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1.0,134170.0,20.079429
2,0.00000,0.000000,0.000000,0.0,0.0,0.0,-0.142707,0.037522,0.148151,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1.0,134170.0,21.789540
3,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,113.0,1.0,27.0,1.0,2.0,1.0,134170.0,17.040536
4,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1.0,55.0,15.742573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1951.0,134170.0,21.565110
99996,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,4955.0,4955.0,9.527477
99997,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,-0.142707,0.037522,0.148151,0.000000,...,11.0,99.0,101.0,1.0,234997.0,1.0,2.0,1.0,134170.0,16.662946
99998,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,36449.0,36449.0,3.169925


In [ ]:
f_input='/content/vldb2021_fauce/training_data/featurization/job-light-content-2500.csv'
f_output='/content/vldb2021_fauce/models_training/datasets/job-light-content-2500.csv'
transform_features_and_labels(f_input, f_output, normalization=False)

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,cardinality
0,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,101.0,101.0,1.0,234997.0,1.0,1.0,1.0,134170.0,16.015938
1,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,101.0,101.0,1.0,234997.0,1.0,2.0,1.0,134170.0,15.967226
2,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,2.0,2.0,1.0,134170.0,10.181152
3,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,99.0,1.0,234997.0,2.0,2.0,1.0,134170.0,18.207100
4,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,2.0,2.0,1.0,134170.0,18.133453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,-0.142707,0.037522,0.148151,-0.002204,...,11.0,100.0,100.0,1.0,234997.0,1.0,1.0,76731.0,76731.0,10.637531
2066,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,-0.142707,0.037522,0.148151,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,1.0,1.0,22017.0,22017.0,14.490161
2067,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,-0.142707,0.037522,0.148151,-0.002204,...,11.0,99.0,113.0,1.0,234997.0,1.0,2.0,1.0,134170.0,30.416777
2068,-0.18052,-0.171498,0.166586,0.0,0.0,0.0,-0.142707,0.037522,0.148151,-0.002204,...,11.0,100.0,100.0,1.0,234997.0,1.0,1.0,2830.0,2830.0,12.412570


## Training

In [4]:
!pip install scikit-learn
!pip install pandas
!pip install tensorflow

In [ ]:
!mkdir -p /content/vldb2021_fauce/models_training/results
!cd /content/vldb2021_fauce/models_training && python train.py --dataset /content/vldb2021_fauce/models_training/datasets/normalized_job-light-content-2500.csv --output results/output.csv --max_iter 600

2024-08-22 22:08:21.069385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 22:08:21.090956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 22:08:21.096783: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 22:08:21.110602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 22:08:22.149412: W tensorflow/comp

In [ ]:
!cd /content/vldb2021_fauce/models_training && python train.py --dataset datasets/JOB_five_joins_training_dataset_log2.csv --output results/output_example.csv --max_iter 300

In [ ]:
!cd /content/vldb2021_fauce/models_training && python train.py --dataset datasets/JobLight-train.csv --output results/JobLight-train.csv

2024-08-23 05:43:11.021695: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-23 05:43:11.039994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 05:43:11.061751: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 05:43:11.068255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 05:43:11.084116: I tensorflow/core/platform/cpu_feature_guar

# Test

In [ ]:
!mkdir /content/vldb2021_fauce/models_training/new/results
!mkdir /content/vldb2021_fauce/models_training/new/models

In [ ]:
!cd /content/vldb2021_fauce/models_training && python train.py --dataset datasets/JobLight-train.csv --output results/JobLight-train.csv --final_test datasets/job-light.csv --max_iter 3000 --batch_size 128 --learning_rate 0.0001

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
2024-08-23 21:18:03.533698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 21:18:03.554285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 21:18:03.560801: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 21:18:04.655866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions f

In [5]:
import os
os.chdir('/content/vldb2021_fauce/models_training')
import tensorflow.compat.v1 as tf
from model import MLPGaussianRegressor
from train import ensemble_mean_var
from utils import DataLoader_RegressionToy
import numpy as np
import pandas as pd
from pandas import DataFrame

tf.disable_v2_behavior()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from train import test_ensemble_on_csv

def create_args():
    # 手动构建一个 args 对象
    class Args:
        def __init__(self):
            self.dataset = 'datasets/JobLight-train.csv'
            self.output = 'results/JobLight-train.csv'
            self.ensemble_size = 10
            self.max_iter = 30000
            self.batch_size = 128
            self.epsilon = 1e-2
            self.alpha = 0.0
            self.learning_rate = 0.0001
            self.grad_clip = 100.0
            self.decay_rate = 0.98
            self.keep_prob = 0.8
            self.test_ratio = 0.1
            self.beta = 1e-3
            self.final_test = 'datasets/job-light.csv'
    args = Args()
    return Args()
model_dir = '/content/vldb2021_fauce/models_training/models/JobLight-train'
args = create_args()
dataLoader = DataLoader_RegressionToy(args)
num_features=50
sizes = [num_features, 128, 256, 512, 512, 2]
ensemble = [MLPGaussianRegressor(args, sizes, 'model'+str(i)) for i in range(args.ensemble_size)]

Instructions for updating:
non-resource variables are not supported in the long term


100000
51
The shape of the data_x is: (100000, 50)
[[18.11457606]
 [20.07942879]
 [21.78954041]
 ...
 [16.66294647]
 [ 3.169925  ]
 [20.84313975]]
self.min_val is: 0.0
self.max_val is: 28.77848829046907
The shape of the train_data_x is: (90000, 50)


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


self.nll_gradients is: Tensor("gradients/MatMul_grad/MatMul:0", shape=(?, 50), dtype=float32)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


self.nll_gradients is: Tensor("gradients_2/MatMul_10_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_4/MatMul_20_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_6/MatMul_30_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_8/MatMul_40_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_10/MatMul_50_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_12/MatMul_60_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_14/MatMul_70_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_16/MatMul_80_grad/MatMul:0", shape=(?, 50), dtype=float32)
self.nll_gradients is: Tensor("gradients_18/MatMul_90_grad/MatMul:0", shape=(?, 50), dtype=float32)


In [ ]:
import os
os.chdir('/content/vldb2021_fauce/models_training')
import tensorflow.compat.v1 as tf
from model import MLPGaussianRegressor
from train import ensemble_mean_var
from utils import DataLoader_RegressionToy
import numpy as np
import pandas as pd
from pandas import DataFrame

tf.disable_v2_behavior()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from train import test_ensemble_on_csv

def create_args():
    # 手动构建一个 args 对象
    class Args:
        def __init__(self):
            self.dataset = 'datasets/JobLight-train.csv'
            self.output = 'results/JobLight-train.csv'
            self.ensemble_size = 10
            self.max_iter = 30000
            self.batch_size = 128
            self.epsilon = 1e-2
            self.alpha = 0.0
            self.learning_rate = 0.0001
            self.grad_clip = 100.0
            self.decay_rate = 0.98
            self.keep_prob = 0.8
            self.test_ratio = 0.1
            self.beta = 1e-3
            self.final_test = 'datasets/job-light.csv'
    args = Args()
    return Args()
model_dir = '/content/vldb2021_fauce/models_training/models/JobLight-train'
args = create_args()
dataLoader = DataLoader_RegressionToy(args)
num_features=50
sizes = [num_features, 128, 256, 512, 512, 2]
ensemble = [MLPGaussianRegressor(args, sizes, 'model'+str(i)) for i in range(args.ensemble_size)]

from train import test_ensemble
from train import test_ensemble_on_csv
test_csv ='/content/vldb2021_fauce/models_training/datasets/JobLight-train.csv'
# test_csv = '/content/vldb2021_fauce/models_training/datasets/job-light.csv'
with tf.Session() as sess:
  for model in ensemble:
    model.load_model(sess, model_dir)
  # test_ensemble(ensemble, sess, dataLoader, args)
  test_ensemble_on_csv(ensemble, sess, dataLoader, args, test_csv)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
test_csv = '/content/vldb2021_fauce/models_training/datasets/job-light.csv'
test_csv.split('/')[-1]

'job-light.csv'

In [8]:
import time
test_csv = '/content/vldb2021_fauce/models_training/datasets/job-light.csv'
with tf.Session() as sess:
  for model in ensemble:
    model.load_model(sess, model_dir)
  start_time = time.time()
  test_ensemble_on_csv(ensemble, sess, dataLoader, args, test_csv)
  execution_time = time.time() - start_time
  print(f"Test {test_csv.split('/')[-1]} Execution time: {execution_time} seconds.")

Model /content/vldb2021_fauce/models_training/models/JobLight-train/model0/model0 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model1/model1 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model2/model2 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model3/model3 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model4/model4 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model5/model5 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model6/model6 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model7/model7 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model8/model8 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model9/model9 restored.
min_val is : 0.0
max_val is : 28.77848829046907
The total number of none nan val

```
Featurization Execution time for 100000 rows: 3381.091047525406 seconds.
Average time: 33.81091047525406 ms

itr: 3000
The shape of the train_data_x is: (90000, 50)
Training time: 183.44 seconds
len(test_ys1): 10000
Test Execution time: 3.30 seconds
```

In [9]:
from train import test_ensemble
from train import test_ensemble_on_csv
test_csv ='/content/vldb2021_fauce/models_training/datasets/job-light-content-2500.csv'
with tf.Session() as sess:
  for model in ensemble:
    model.load_model(sess, model_dir)
  start_time = time.time()
  test_ensemble_on_csv(ensemble, sess, dataLoader, args, test_csv)
  execution_time = time.time() - start_time
  print(f"Test {test_csv.split('/')[-1]} Execution time: {execution_time} seconds.")

Model /content/vldb2021_fauce/models_training/models/JobLight-train/model0/model0 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model1/model1 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model2/model2 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model3/model3 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model4/model4 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model5/model5 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model6/model6 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model7/model7 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model8/model8 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model9/model9 restored.
min_val is : 0.0
max_val is : 28.77848829046907
The total number of none nan val

In [10]:
f_csv = '/content/vldb2021_fauce/models_training/datasets/JobLight-train.csv'
df = pd.read_csv(f_csv)
df_last_10000 = df.tail(10000)
df_last_10000.to_csv('/content/vldb2021_fauce/models_training/datasets/JobLight-train-last10k.csv', index=False)

In [14]:
f_csv = '/content/vldb2021_fauce/training_data/raw_data/JobLight-train.csv'
df = pd.read_csv(f_csv, sep='#')
df_last_10000 = df.tail(10000)
df_last_10000.to_csv('/content/vldb2021_fauce/training_data/raw_data/JobLight-train-last10k.csv', index=False)

In [11]:
from train import test_ensemble
from train import test_ensemble_on_csv
test_csv ='/content/vldb2021_fauce/models_training/datasets/JobLight-train-last10k.csv'
with tf.Session() as sess:
  for model in ensemble:
    model.load_model(sess, model_dir)
  start_time = time.time()
  test_ensemble_on_csv(ensemble, sess, dataLoader, args, test_csv)
  execution_time = time.time() - start_time
  print(f"Test {test_csv.split('/')[-1]} Execution time: {execution_time} seconds.")

Model /content/vldb2021_fauce/models_training/models/JobLight-train/model0/model0 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model1/model1 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model2/model2 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model3/model3 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model4/model4 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model5/model5 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model6/model6 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model7/model7 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model8/model8 restored.
Model /content/vldb2021_fauce/models_training/models/JobLight-train/model9/model9 restored.
min_val is : 0.0
max_val is : 28.77848829046907
The total number of none nan val

## Test Summary

In [ ]:
import pandas as pd
df_error = pd.read_csv('/content/vldb2021_fauce/models_training/results/JobLight-train.csv')
stats = df_error['error0'].agg(['min','max','mean', 'var'])
quantiles = df_error['error0'].quantile([0.50, 0.75, 0.90, 0.95, 0.99])
combined_stats =  pd.concat([stats, quantiles])
combined_stats = combined_stats.round(2)
combined_stats

,error0
min,1.00
max,394803.00
mean,211.47
var,24643478.76
0.5,2.16
0.75,6.66
0.9,28.17
0.95,100.61
0.99,1671.13


In [ ]:
import pandas as pd
df_error = pd.read_csv('/content/vldb2021_fauce/models_training/results/trained_on_JobLight-train_tested_on_JobLight-train.csv')
stats = df_error['error'].agg(['min','max','mean', 'var'])
quantiles = df_error['error'].quantile([0.50, 0.75, 0.90, 0.95, 0.99])
combined_stats =  pd.concat([stats, quantiles])
combined_stats = combined_stats.round(2)
combined_stats

,error
min,1.00
max,545886.00
mean,118.01
var,9878714.59
0.5,1.96
0.75,5.78
0.9,21.94
0.95,72.78
0.99,1046.02


In [ ]:
import pandas as pd
df_error = pd.read_csv('/content/vldb2021_fauce/models_training/results/trained_on_JobLight-train_tested_on_job-light.csv')
stats = df_error['error'].agg(['min','max','mean', 'var'])
quantiles = df_error['error'].quantile([0.50, 0.75, 0.90, 0.95, 0.99])
combined_stats =  pd.concat([stats, quantiles])
combined_stats = combined_stats.round(2)
combined_stats

,error
min,1.06
max,2166.62
mean,61.47
var,76647.85
0.5,3.18
0.75,12.59
0.9,38.35
0.95,234.65
0.99,1165.00


In [ ]:
import pandas as pd
df_error = pd.read_csv('/content/vldb2021_fauce/models_training/results/trained_on_JobLight-train_tested_on_job-light-content-2500.csv')
stats = df_error['error'].agg(['min','max','mean', 'var'])
quantiles = df_error['error'].quantile([0.50, 0.75, 0.90, 0.95, 0.99])
combined_stats =  pd.concat([stats, quantiles])
combined_stats = combined_stats.round(2).apply(lambda x: f'{x:.2f}')
combined_stats

,error
min,1.00
max,2030750.00
mean,1050.73
var,1992287425.74
0.5,4.69
0.75,18.49
0.9,73.52
0.95,205.19
0.99,1369.19


In [12]:
import pandas as pd
df_error = pd.read_csv('/content/vldb2021_fauce/models_training/results/trained_on_JobLight-train_tested_on_JobLight-train-last10k.csv')
stats = df_error['error'].agg(['min','max','mean', 'var'])
quantiles = df_error['error'].quantile([0.50, 0.75, 0.90, 0.95, 0.99])
combined_stats =  pd.concat([stats, quantiles])
combined_stats = combined_stats.round(2).apply(lambda x: f'{x:.2f}')
combined_stats

,error
min,1.00
max,394804.00
mean,211.47
var,24643562.11
0.5,2.16
0.75,6.66
0.9,28.17
0.95,100.61
0.99,1671.13


## Test on Postgres

In [ ]:
#